In [1]:
import asyncio
import pybotters
import pandas as pd
from loguru import logger

import json
import requests
import pandas as pd
import datetime
import time

In [2]:
# e.g. https://api.cryptowat.ch/markets/:exchange/:pair/ohlc
def generate_path(
        exchange: str,
        pair: str,
        min: int,
        unixTime: int) -> str:
    
    return f'https://api.cryptowat.ch/markets/{exchange}/{pair}/ohlc?periods={min}&after={unixTime}'

# print(generate_path(1675368565))

In [3]:
def str_to_timestamp(s: str) -> int:

    # str → datetime e.g.'2022-05-27 10:11:23'
    dt = datetime.datetime.strptime(s, "%Y-%m-%d %H:%M:%S")

    # datetime → timestamp
    ts = datetime.datetime.timestamp(dt)

    return ts

ts = str_to_timestamp('2022-05-27 10:11:23')
print(ts)

1653613883.0


In [4]:
async def main(candle: int, before: int, size: int) -> pd.DataFrame:
    async with pybotters.Client() as client:
        url="https://api.cryptowat.ch/markets/bitflyer/btcfxjpy/ohlc?periods="
        candle=candle*60
        period=("%s"%(candle))
        period=[period]
        beforeurl="&before="
        afterurl="&after="
        # The maximum size of data is 6000 depending on a timing (sometimes you get less than 6000)
        after= before-(candle*size)
        URL=("%s%s%s%s%s%s"%(url,candle,afterurl,after,beforeurl,before))
        # e.g https://api.cryptowat.ch/markets/bitflyer/btcjpy/ohlc?periods=86400&after=1483196400

        response = await client.get(URL)
        body = await response.json()
        res = body['result']
        
        data = []
        for i in period:
            row = res[i]
            for column in row:
                if column[4] != 0:
                    column = column[0:6]
                    data.append(column)

        date = [price[0] for price in data]
        op = [int(price[1]) for price in data]
        hi = [int(price[2]) for price in data]
        lo = [int(price[3]) for price in data]
        cl = [int(price[4]) for price in data]
        date_datetime = map(datetime.datetime.fromtimestamp, date)
        dti = pd.DatetimeIndex(date_datetime)
        df = pd.DataFrame({"op" : op, "hi" : hi, "lo": lo, "cl" : cl}, index=dti)
        
        return df

In [5]:
if __name__ == '__main__':
    try:
        df = await main(15, 1675368565, 6000)
        print(df)
    except KeyboardInterrupt:
        pass

                          op       hi       lo       cl
2022-12-03 16:45:00  2299477  2300635  2297938  2300135
2022-12-03 17:00:00  2300092  2300964  2299067  2299972
2022-12-03 17:15:00  2299717  2299949  2296876  2297998
2022-12-03 17:30:00  2298287  2299335  2296649  2298026
2022-12-03 17:45:00  2298046  2298289  2295326  2295604
...                      ...      ...      ...      ...
2023-02-03 04:00:00  3102032  3102463  3095000  3095964
2023-02-03 04:15:00  3095864  3095864  3069010  3077738
2023-02-03 04:30:00  3076769  3078635  3059114  3071172
2023-02-03 04:45:00  3070672  3073948  3055657  3059888
2023-02-03 05:00:00  3060484  3070066  3058927  3067454

[5902 rows x 4 columns]
